<a href="https://colab.research.google.com/github/acmilannesta/Bert-embedding/blob/master/GoogleQA_tfhub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!nvidia-smi

Tue Dec 10 14:44:16 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.36       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    35W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
# !pip install keras-bert
!wget https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1?tf-hub-format=compressed
!mkdir bert_base
!tar -xzf 1?tf-hub-format=compressed -C bert_base

# wget https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip
# unzip wwm_uncased_L-24_H-1024_A-16.zip
# wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-24_H-1024_A-16.zip
# unzip uncased_L-24_H-1024_A-16.zip

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
import os, gc
import codecs
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import Callback
from keras.models import Model, load_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, GroupKFold, KFold
from scipy.stats import spearmanr
from tqdm import tqdm
import tensorflow_hub as hub
from tokenizer import Tokenizer 
# from keras_bert import load_trained_model_from_checkpoint, Tokenizer, AdamWarmup, calc_train_steps, get_custom_objects
from hyperopt import fmin, hp, tpe, STATUS_OK, Trials


In [0]:
data_path = 'drive/My Drive/GoogleQA/Data/'
train = pd.read_csv(data_path+'train.csv')
test = pd.read_csv(data_path+'test.csv')
sub = pd.read_csv(data_path+'sample_submission.csv')

# train_q = pd.read_csv(data_path+'train_q.csv')
# test_q = pd.read_csv(data_path+'test_q.csv')
# train_a = pd.read_csv(data_path+'train_answer.csv')
# test_a = pd.read_csv(data_path+'test_answer.csv')

# df_aux = pd.concat([train[['host', 'category']], test[['host', 'category']]], 0)
# aux1 = pd.get_dummies(df_aux['category'], drop_first=True)
# aux2 = pd.get_dummies(df_aux['host'].apply(lambda x: x.split('.')[0]), drop_first=True)
# aux = pd.concat([aux1, aux2], 1)
# train_aux = pd.concat([train, aux.iloc[:len(train), ]], 1)
# test_aux =  pd.concat([test, aux.iloc[len(train):, ]], 1)

In [0]:
MAXLEN = 256 #@param {type:"slider", min:128, max:512, step:32}
BATCH_SIZE = 4 #@param {type:'slider', min:1, max:32, step:1}
NUM_EPOCHS = 3
NUM_CLASSES = 30
LR = 5e-5
MIN_LR = 0
# OUTPUT_TRAIN = 'train_bert_ipredcv1415_oof.csv'
# OUTPUT_TEST = 'test_bert_large.npy'
model_path = 'uncased_L-12_H-768_A-12' #@param ['uncased_L-12_H-768_A-12', 'wwm_uncased_L-24_H-1024_A-16', 'uncased_L-24_H-1024_A-16']
# target_q_col = train.columns.tolist()[11:32]
# target_a_col = train.columns.tolist()[32:42]
target_col = train.columns.tolist()[11:42]

In [0]:
token_dict = {}
with codecs.open(os.path.join('bert_base/assets/vocab.txt'), 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)
tokenizer = Tokenizer(token_dict)

In [0]:
def convert_data(data_df, branch='training'):
    data_df = data_df.reset_index(drop=True)
    global tokenizer
    global MAXLEN
    global target_col
    # global target_a_col
    # global target_q_col
    q_title, q_body, answer = [], [], []
    for i in tqdm(range(len(data_df))):
        answer_ids, _ = tokenizer.encode(data_df.loc[i, 'answer'])
        answer.append(answer_ids)                 
        q_title_ids, _ = tokenizer.encode(data_df.loc[i, 'question_title'])
        q_title.append(q_title_ids)
        q_body_ids, _ = tokenizer.encode(data_df.loc[i, 'question_body'])
        q_body.append(q_body_ids)
    if branch == 'training':
        targets = data_df[target_col]
        return [q_title, q_body, answer], np.array(targets)
    else:
        return [q_title, q_body, answer]



In [0]:
"""## Data Generator"""
def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = min(max(L), MAXLEN)
    return np.array([np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x[:ML] for x in X])

def get_masks(X, padding=0):
    L = [len(x) for x in X]
    ML = min(max(L), MAXLEN)
    return np.array([np.concatenate([[1]*len(x), [padding] * (ML - len(x))]) if len(x) < ML else [1]*ML for x in X])


class data_generator:
    def __init__(self, data, batch_size=BATCH_SIZE, branch='train'):
        self.data = data
        self.batch_size = batch_size
        self.branch = branch
        # self.q_a = q_a
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1

    def __len__(self):
        return self.steps

    def __iter__(self):
        while True:
            if self.branch == 'train':
                np.random.shuffle(self.data)
            for i in range(self.steps):
                d = self.data[i * self.batch_size: (i + 1) * self.batch_size]
                X1 = seq_padding([x[0] for x in d])
                X2 = get_masks([x[0] for x in d])    
                X3 = np.zeros_like(X1)

                X4 = seq_padding([x[1] for x in d])
                X5 = get_masks([x[1] for x in d])    
                X6 = np.zeros_like(X4)

                X7 = seq_padding([x[2] for x in d])
                X8 = get_masks([x[2] for x in d])    
                X9 = np.zeros_like(X7)

                if self.branch == 'test':
                    # aux = np.array([x[3] for x in d])
                    yield [X1, X2, X3, X4, X5, X6, X7, X8, X9]
                else:
                    Y = np.array([x[3] for x in d])
                    # aux = np.array([x[4] for x in d])
                    yield [X1, X2, X3, X4, X5, X6, X7, X8, X9], Y



In [0]:
def model_build():
    global NUM_CLASSES
    global BATCH_SIZE
    global NUM_EPOCHS
    global MIN_LR
    global LR
    global MAXLEN

    # bert_model = load_trained_model_from_checkpoint(
    #     os.path.join(model_path, 'bert_config.json'),
    #     os.path.join(model_path, 'bert_model.ckpt'),
    #     seq_len = MAXLEN,
    #     trainable=True,
    # )
    bert_layer = hub.KerasLayer("bert_base", trainable=True)

    q_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="q_input_word_ids")
    q2_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="q_input_masks")
    q3_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="q_segment_ids")

    qb_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="qb_input_word_ids")
    qb2_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="qb_input_masks")
    qb3_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="qb_segment_ids")

    a_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="a_input_word_ids")
    a2_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="a_input_masks")
    a3_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="a_segment_ids")

    _, q_inputs  = bert_layer([q_in, q2_in, q3_in])
    q_outputs = keras.layers.Lambda(lambda x: x[:, 0])(q_inputs)

    _, qb_inputs  = bert_layer([qb_in, qb2_in, qb3_in])
    qb_outputs = keras.layers.Lambda(lambda x: x[:, 0])(qb_inputs)

    _, a_inputs  = bert_layer([a_in, a2_in, a3_in])
    a_outputs = keras.layers.Lambda(lambda x: x[:, 0])(a_inputs)

    dense = keras.layers.concatenate([q_outputs, qb_outputs, a_outputs])
    outputs = keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(dense)
    model = keras.Model([q_in, q2_in, q3_in, qb_in, qb2_in, qb3_in, a_in, a2_in, a3_in], outputs)

    # decay_steps, warmup_steps = calc_train_steps(
    #     len_train,
    #     batch_size=BATCH_SIZE,
    #     epochs=NUM_EPOCHS,
    # )

    model.compile(
        loss='binary_crossentropy',
        optimizer=keras.optimizers.Adam(LR),
        # optimizer=AdamWarmup(
        #     decay_steps=decay_steps,
        #     warmup_steps=warmup_steps,
        #     lr=LR,
        #     min_lr=MIN_LR,
        #     ))
    )
    del bert_layer
    gc.collect()
    return model


## train model

In [21]:
test_x = convert_data(test, branch='testing')
pred = np.zeros((len(test), NUM_CLASSES))
kf = KFold(n_splits=5, shuffle=True, random_state=0)
idx = [x for x in kf.split(train)]

def compute_spearmanr(trues, preds):
    rhos = []
    for col_trues, col_pred in zip(trues.T, preds.T):
        rhos.append(spearmanr(col_trues, col_pred + np.random.normal(0, 1e-7, col_pred.shape[0])).correlation)
    return np.nanmean(rhos)

class IntervalEval(keras.callbacks.Callback):
    def __init__(self, val_data, test_data, label):
        global NUM_EPOCHS
        super(keras.callbacks.Callback, self).__init__()
        self.val_data = val_data
        self.test_data = test_data
        self.label = label
        self.total_epochs = NUM_EPOCHS
        self.test_pred = []
        self.score = []
    def on_epoch_end(self, epoch, logs={}):
        # if epoch == 3:
        val_pred = self.model.predict_generator(self.val_data.__iter__(), len(self.val_data), verbose=1)
        score = compute_spearmanr(self.label, val_pred)
        self.score.append(score)
        print('Spearman - {:.5f}'.format(score))
        if epoch>=self.total_epochs-2:
            print("Testing Pred")
            test_pred = self.model.predict_generator(self.test_data.__iter__(), len(self.test_data), verbose=1)
            self.test_pred.append(test_pred)
        if epoch==self.total_epochs-1:
            if self.score[-2]<=self.score[-1]:
                self.test_pred = self.test_pred[0] * 0.3 + self.test_pred[1] * 0.7
            else:
                self.test_pred = self.test_pred[0] * 0.7 + self.test_pred[1] * 0.3

100%|██████████| 476/476 [00:01<00:00, 259.97it/s]


In [0]:
for i, (tr_idx, val_idx) in enumerate(idx[0:1], 1):
    print('\nFold - {:}\n'.format(i))
    tr, val = train.loc[tr_idx], train.loc[val_idx]
    tr_x, tr_y = convert_data(tr)
    val_x, val_y = convert_data(val)
    
    model = model_build()

    train_D = data_generator(list(zip(tr_x[0], tr_x[1], tr_x[2], tr_y)))
    valid_D = data_generator(list(zip(val_x[0], val_x[1], val_x[2], val_y)), branch='valid')
    test_D = data_generator(list(zip(test_x[0], test_x[1], test_x[2])), branch='test')
    ieval = IntervalEval(val_data=valid_D, label=val_y, test_data=test_D)
    model.fit_generator(
        train_D.__iter__(),
        steps_per_epoch=len(train_D),
        epochs=NUM_EPOCHS,
        callbacks = [ieval]
    )
    # oof_pred = model.predict_generator(valid_D.__iter__(), len(valid_D), verbose=1)
    # train_aug.loc[val_idx, 'oof_pred'] = np.argmax(oof_pred, 1)
    # print('oof - {:} f1_score - {:.4f}'.format(i, spearmanr(val_y, np.argmax(oof_pred, 1), average='weighted')))

    pred+= ieval.test_pred / kf.get_n_splits()
    # np.save(OUTPUT_TEST, pred)
    # s3.upload_file(Filename=OUTPUT_TEST, Bucket='acmilannesta', Key='large/'+OUTPUT_TEST)

    # model_file = 'model-oof-'+str(i)+'.h5'
    # model.save('model.h5')

    del model
    gc.collect()

Epoch 1/3


In [60]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
q_input_word_ids (InputLayer)   [(None, None)]       0                                            
__________________________________________________________________________________________________
q_input_masks (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
q_segment_ids (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
qb_input_word_ids (InputLayer)  [(None, None)]       0                                            
____________________________________________________________________________________________

In [0]:
np.min(a[0])

5.1090916e-05

In [0]:
sub.iloc[:, 1:] = pred
sub.to_csv('drive/My Drive/GoogleQA/Submissions/sub_1.csv', index=False)

array([[0.06384172, 0.04833349, 0.01118633, ..., 0.00183944, 0.0543016 ,
        0.0663632 ],
       [0.06220622, 0.0320207 , 0.00025662, ..., 0.00794972, 0.00680789,
        0.06667056],
       [0.06475838, 0.04486672, 0.00184575, ..., 0.00371491, 0.07216934,
        0.06733149],
       ...,
       [0.06101127, 0.02670371, 0.00171441, ..., 0.01432018, 0.02452305,
        0.07175547],
       [0.05496352, 0.04671385, 0.00131091, ..., 0.00379235, 0.05892152,
        0.05909493],
       [0.06234278, 0.03923614, 0.00073717, ..., 0.00816545, 0.01689972,
        0.062443  ]])